# De Finale Modellen Creeren en hier een verklaring bij geven.
Hierbij zijn voor Thomas en Erwin 2 modellen gemaakt: met en zonder WOX.


Indien WOX data beschikbaar is, gebruiken wij het wox model. Hier wordt uitgegaan van een optimaal iteraties at 75. Voor het niet WOX model wordt een aantal iteraties van X gebruikt. 
Deze waardes zijn om overfitting te voorkomen. Wij willen het model alleen dingen leren die te generaliseren zijn. 


De WOX en niet WOX modellen worden samengevoegd om tot een Ensemble te komen.

Via SHAP values worden verklaringen gegeven voor waarom een voorspelling wordt gedaan. 

Per buurt willen wij hierdoor per emsemble model (= wox/geen wox samengevoegd)
* Voorspelling TK 
* Voorspelling EE 
* Confidence TK voorspellingen 
* Confidence EE voorspellingen
* Feature Importance TK
* Feature Importance EE

---------------
In deze notebook creeren wij de 4 finale modellen

In [1]:
# importeren packages
import pandas as pd
import numpy as np
import xgboost
import pickle
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import xgboost
from xgboost import plot_importance
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold, cross_val_score

In [2]:
# setten source path: later moet alle brondata naar datalake
source_path ="P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"
# alle tussentijdse data voorlopig in dezelfde map op een schijf opslaan in
save_path = "P:\\DATASCIENCE\\Aantrekkelijkheid buurten\\Resultaten\\"

In [3]:
# inladen bestand met alle buurten en onafhankelijke variabelen als all
buurtendata = pd.read_csv(source_path + 'buurten.csv', encoding='utf-16')
buurtendata.rename(columns={"RLBBEV18":"RLBBEV",'KL18': "KL", 
               'RLBRMTR18': 'RLBRMTR', 'RLBWON18':'RLBWON' , 'RLBBEV18':'RLBBEV', 'RLBVRZ18':'RLBVRZ', 'RLBVEI18': 'RLBVEI', 'RLBFYS18': 'RLBFYS',}, inplace=True) 

In [4]:
# inladen bestand met scores als sc
sc = pd.read_excel(source_path + 'buurten voor experts 2e ronde.xlsx', sheet_name='buurten')
# alleen selecteren kolommen voor scores en oordelen
sc = sc[['buurtcode','EE','TK']]

In [5]:
# merge op buurtcode (die van 2019)
df = pd.merge(sc, buurtendata, on='buurtcode')

## Trainen van de Modellen voor Erwin
### WOX

In [6]:
# minus de kolommen die niet in de 2016 dataset aanwezig zijn
               
## EE wox filterd.               
feature_cols = 	['oadh', 'KL', 'RLBRMTR', 'RLBBEV', 'RLBVRZ', 'RLBFYS', 'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'aandeelMH19', 'auto30', 'fiets', 'wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2', 'woxm2Gem', 'woxdelta5jrWGabs', 'woxdelta5jrWGrel', 'woxm2WG']

       
X = df[feature_cols]
y = df['EE']

In [7]:
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=70, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=70, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [8]:
fi = pd.DataFrame(model.feature_importances_,X.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
woxm2Gem,0.234091,woxm2Gem
woxm2,0.181392,woxm2
woxww,0.160898,woxww
woxdelta5jr,0.051709,woxdelta5jr
woxm2WG,0.050305,woxm2WG
wox,0.035743,wox
KL,0.033038,KL
oadh,0.023940,oadh
woxdelta1jr,0.023507,woxdelta1jr
RLBBEV,0.022792,RLBBEV


In [9]:
# Het opslaan van het model
file_name = "../models/final_model_EE_wox.pkl"
pickle.dump(model, open(file_name, "wb"))

### Volledige dataset voorspelling
Hier testen we de accuracy als wij een getraind model gebruiken om de volledige dataset te voorspellen.

In [10]:
# Wel enkel de kolommen nodig waarop ons model is getraind
X_pred = buurtendata[feature_cols]
X_pred = pd.DataFrame(X_pred)
X_pred.columns = feature_cols

In [11]:
final_predictions_df = pd.DataFrame(buurtendata['buurtcode'])
final_predictions_df['ee_wox'] = model.predict(X_pred)
X_pred['ee_wox'] = model.predict(X_pred)

In [12]:
final_predictions_df.head()

,buurtcode,ee_wox
0,30000,c
1,30001,c
2,30002,c
3,30007,c
4,30008,c


In [13]:
# Hier koppelen we de 'echte' waardes hieraan om de accuracy te voorspellen. Dit is een validatie stap
X_pred['buurtcode'] = buurtendata['buurtcode']
X_pred = X_pred.merge(sc, how='left', on ='buurtcode')
temp = X_pred[X_pred['EE'].notnull()][['EE', 'ee_wox']]
accuracy_score(temp['ee_wox'].values,temp['EE'].values)


0.9970414201183432

## Model zonder wox EE

In [14]:
## EE wox filterd.               
feature_cols = 		['oadh', 'KL', 'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBFYS', 'AantalLevensm3', 'AantalCafe3', 'AantalRest3', 'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 'hh19inkMidden', 'hh30inkMidden', 'hh19inkHoog', 'hh30inkHoog', 'aandeelMH19', 'auto30', 'auto60', 'ov30', 'fiets', 'banen', 'd_kern']
       
X = df[feature_cols]
y = df['EE']

# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [15]:
fi = pd.DataFrame(model.feature_importances_,X.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
d_kern,0.204485,d_kern
AantalPodiumk10,0.107528,AantalPodiumk10
AantalMusea10,0.063996,AantalMusea10
KL,0.063778,KL
fiets,0.053097,fiets
RLBVRZ,0.042387,RLBVRZ
ov30,0.036658,ov30
hh19inkHoog,0.035763,hh19inkHoog
RLBBEV,0.035332,RLBBEV
RLBWON,0.035287,RLBWON


In [16]:
# Het opslaan van het model
file_name = "../models/final_model_EE_nowox.pkl"
pickle.dump(model, open(file_name, "wb"))

In [17]:
# Wel enkel de kolommen nodig waarop ons model is getraind
X_pred = buurtendata[feature_cols]
X_pred = pd.DataFrame(X_pred)
X_pred.columns = feature_cols

In [18]:
#final_predictions_df = pd.DataFrame(buurtendata['buurtcode'])
final_predictions_df['ee_nowox'] = model.predict(X_pred)
X_pred['ee_nowox'] = model.predict(X_pred)

In [19]:
# Hier koppelen we de 'echte' waardes hieraan om de accuracy te voorspellen. Dit is een validatie stap
X_pred['buurtcode'] = buurtendata['buurtcode']
X_pred = X_pred.merge(sc, how='left', on ='buurtcode')
temp = X_pred[X_pred['EE'].notnull()][['EE', 'ee_nowox']]
accuracy_score(temp['ee_nowox'].values,temp['EE'].values)

0.9970414201183432

In [20]:
final_predictions_df.head()

,buurtcode,ee_wox,ee_nowox
0,30000,c,c
1,30001,c,b
2,30002,c,c
3,30007,c,a
4,30008,c,c


## Thomas model
### Met wox

In [21]:
# Hier hebben wij een dataset nodig met de labels van Thomas. 
feature_cols = ['oadh', 'KL', 'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'RLBVEI', 'RLBFYS', 'AantalLevensm3', 'AfstandHuisarts', 'AantalCafe3', 'AantalRest3', 'AfstandZwembad', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 'aandeelMH19', 'auto30', 'ov30', 'fiets', 'wox', 'woxdelta1jr', 'woxdelta5jr', 'woxww', 'woxm2', 'woxm2Gem', 'woxdelta5jrWGabs', 'woxdelta5jrWGrel', 'woxm2WG', 'banen']


X = df[feature_cols]
y = df['TK']

In [22]:
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=60, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=60, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [23]:
fi = pd.DataFrame(model.feature_importances_,X.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
AantalBios5,0.099407,AantalBios5
woxdelta5jr,0.093094,woxdelta5jr
AantalCafe3,0.080370,AantalCafe3
fiets,0.078096,fiets
wox,0.054561,wox
woxdelta5jrWGabs,0.048609,woxdelta5jrWGabs
woxww,0.045080,woxww
AantalLevensm3,0.040006,AantalLevensm3
woxm2WG,0.039542,woxm2WG
RLBVRZ,0.037079,RLBVRZ


In [24]:
# Opslaan van het model
file_name = "../models/final_model_TK_wox.pkl"
pickle.dump(model, open(file_name, "wb"))

In [25]:
# Wel enkel de kolommen nodig waarop ons model is getraind
X_pred = buurtendata[feature_cols]
X_pred = pd.DataFrame(X_pred)
X_pred.columns = feature_cols

#opslaan finale voorspelling van dit model
final_predictions_df['TK_wox'] = model.predict(X_pred)

final_predictions_df.head()

,buurtcode,ee_wox,ee_nowox,TK_wox
0,30000,c,c,c
1,30001,c,b,c
2,30002,c,c,c
3,30007,c,a,c
4,30008,c,c,c


In [26]:
# Hier koppelen we de 'echte' waardes hieraan om de accuracy te voorspellen. Dit is een validatie stap
X_pred['TK_wox'] = model.predict(X_pred)
X_pred['buurtcode'] = buurtendata['buurtcode']
X_pred = X_pred.merge(sc, how='left', on ='buurtcode')
temp = X_pred[X_pred['TK'].notnull()][['TK', 'TK_wox']]
accuracy_score(temp['TK_wox'].values,temp['TK'].values)

1.0

## Thomas zonder WOX
Hierbij wordt gebruik gemaakt van andere kolommmen. 

In [27]:
feature_cols = ['oadh', 'KL', 'RLBRMTR', 'RLBWON', 'RLBBEV', 'RLBVRZ', 'AantalLevensm3', 'AantalCafe3', 'AantalRest3', 'AantalMusea10', 'AantalPodiumk10', 'AantalBios5', 'hh19inkHoog', 'auto30', 'auto60', 'ov30', 'ov60', 'fiets', 'banen']
X = df[feature_cols]
y = df['TK']

In [28]:
# Ideaal bleek rond de 30 iteraties
# maak object met geoptimaliseerde hyperparameters
model = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=None, n_estimators=55, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2,
              silent=None, subsample=0.8, verbosity=1)

# en fit op data
model.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=3, missing=nan, monotone_constraints=None,
              n_estimators=55, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=2, silent=None,
              subsample=0.8, tree_method=None, validate_parameters=False,
              verbosity=1)

In [29]:
# Model opslaan
file_name = "../models/final_model_TK_nowox.pkl"
pickle.dump(model, open(file_name, "wb"))

In [30]:
fi = pd.DataFrame(model.feature_importances_,X.columns)
fi.columns = ['feature_score']
fi.sort_values('feature_score', ascending=False, inplace=True)
fi['feature'] = fi.index
fi

,feature_score,feature
AantalCafe3,0.131060,AantalCafe3
fiets,0.100461,fiets
RLBVRZ,0.066856,RLBVRZ
banen,0.066754,banen
AantalMusea10,0.063557,AantalMusea10
auto60,0.061545,auto60
AantalBios5,0.057311,AantalBios5
KL,0.052533,KL
ov60,0.050638,ov60
RLBBEV,0.045789,RLBBEV


In [31]:
# Wel enkel de kolommen nodig waarop ons model is getraind
X_pred = buurtendata[feature_cols]
X_pred = pd.DataFrame(X_pred)
X_pred.columns = feature_cols

#opslaan finale voorspelling van dit model
final_predictions_df['TK_nowox'] = model.predict(X_pred)

final_predictions_df.head()

,buurtcode,ee_wox,ee_nowox,TK_wox,TK_nowox
0,30000,c,c,c,c
1,30001,c,b,c,c
2,30002,c,c,c,c
3,30007,c,a,c,c
4,30008,c,c,c,c


In [32]:
# Hier koppelen we de 'echte' waardes hieraan om de accuracy te voorspellen. Dit is een validatie stap
X_pred['TK_nowox'] = model.predict(X_pred)
X_pred['buurtcode'] = buurtendata['buurtcode']
X_pred = X_pred.merge(sc, how='left', on ='buurtcode')
temp = X_pred[X_pred['TK'].notnull()][['TK', 'TK_nowox']]
accuracy_score(temp['TK_nowox'].values,temp['TK'].values)

0.9970414201183432

In [33]:
# Opslaan van de predictie resultaten
final_predictions_df.to_csv('../2019model_predictions.csv')

## Het gebruiken en testen van onze getrainde modellen

In [34]:
# laad model
model_2019_EE = pickle.load(open("../models/final_model_EE_wox.pkl", "rb"))
model_2019_TK = pickle.load(open("../models/final_model_TK_wox.pkl", "rb"))
model_2019_EE_nw = pickle.load(open("../models/final_model_EE_nowox.pkl", "rb"))
model_2019_TK_nw = pickle.load(open("../models/final_model_TK_nowox.pkl", "rb"))